## Reddit/Quora scraping

#### Requirements:

In [1]:
# pip install praw

In [2]:
import praw #Reddit scraper
import re #Regular expressions
import random #To select a comment

#### Reddit API use your own credentials:

In [3]:
reddit = praw.Reddit(
    client_id="LSZOduypQfuIUw",
    client_secret="4-RBX3XtMd52Q3FYRuuXb3I-9IhatQ",
    password="H4tutspa44word",
    user_agent="Tutorial",
    username="Hktuts",
)

#### Check if we're set up correctly (should return our reddit username)

In [4]:
print(reddit.user.me())

Hktuts


#### URL to the thread we want to scrape comments from:

In [5]:
url = "https://www.reddit.com/r/worldnews/comments/mo0vvc/norway_prime_minister_fined_1715_for_breaking/"
submission = reddit.submission(url=url)

#### Scraping reddit comments:

In [8]:
comments = []

submission.comments.replace_more(limit=None)

for comment in submission.comments.list():
    comments.append(comment.body)

print("Number of comments scraped from thread: " + str(len(comments)))

Number of comments scraped from thread: 1989


#### Picking a random comment from the first 50:

In [18]:
print(comments[(random.randint(1, 50))])

Yes! Those in authority should be held to a higher standard not a lower one.
